In [ ]:
# -*- coding:utf-8 -*-

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# %%time

# import os

# filePath = r'D:\data'

# fileMap = {}
# size = 0

# # 遍历filePath下的文件、文件夹（包括子目录）
# for parent, dirnames, filenames in os.walk(filePath):
#     for dirname in dirnames:
# #         print('parent is %s, dirname is %s' % (parent, dirname))
#         pass

#     for filename in filenames:
# #         print('parent is %s, filename is %s' % (parent, filename))
# #         print('the full name of the file is %s' % os.path.join(parent, filename))
        
#         size = os.path.getsize(os.path.join(parent, filename))
#         fileMap.setdefault(os.path.join(parent, filename), size)

# filelist = sorted(fileMap.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# %%time

# def df():
#     for filename, size in filelist[:400]:
#         reader = pd.read_csv(filename, index_col=False, iterator=True)
#         loop = True
#         chunkSize = 2000000
#         chunks = []
#         while loop:
#             try:
#                 chunk = reader.get_chunk(chunkSize)
#                 chunk = chunk[['Value']]#['2018-07-12 00:00:00':'2018-07-19 00:00:00']
#                 col = filename.split('.')[-2].split('\\')[-1]
#                 chunk.rename(columns={'Value':col}, inplace=True)
#                 chunks.append(chunk)
#             except StopIteration:
#                 loop = False
#                 print ("Iteration is stopped.")
#         df = pd.concat(chunks, ignore_index=True, axis=0)
#         yield df

# dfs = pd.concat(df(), axis=1)

# for filename, size in filelist[:1]:
#     reader = pd.read_csv(filename, index_col=False)
#     dfs.index = reader['Time']

In [ ]:
import struct


# 读取并解析二进制文件
def rbf(path):
    fd = open(path, 'rb')
    sigs = []
    sigstime = []
    while fd:
        buf = fd.read(60)
        if len(buf) == 0:
            break

        sig_status = struct.unpack('B',buf[:1])  # 烟支状态
        spare_0 = struct.unpack('B',buf[1:2])
        bo_number = struct.unpack('L',buf[2:6])  # 烟支序号
        bo_time = struct.unpack('8H',buf[6:22])  # 烟支生产时间
        bo_time = list(bo_time)
        del bo_time[2]  # 删除本周的第几天
        day = '-'.join([str(i) for i in bo_time[:3]])
        hour = ':'.join([str(i) for i in bo_time[3:-1]])
        bo_time = []
        bo_time.append(day + ' ' + hour)  # 组合成'2018-12-14 8:18:28'的形式
        bo_srm_Weight = struct.unpack('H',buf[22:24])  # 重量
        bo_srm_xsd = struct.unpack('f',buf[24:28])  # 吸丝带位置
        bo_cis = struct.unpack('6H',buf[28:40])  # 吸阻，通风度，漏气，松头，光学外观，圆周值
        sig = sig_status+spare_0+bo_number+bo_srm_Weight+bo_srm_xsd+bo_cis
        sig_time = bo_time
        sigs.append(sig)
        sigstime.append(sig_time)
    fd.close()
    return sigs, sigstime

In [ ]:
# 根据解析出来的列表数据生成DataFrame数据
def gendf(path, rule='1s'):
    sigs, sigstime = rbf(path)

    df = pd.DataFrame(sigs)
    df.columns = ['烟支状态','spare_0','烟支序号','重量','吸丝带位置','吸阻','通风度','漏气','松头','光学外观','圆周值']

    time = pd.DataFrame(sigstime)
    time.columns = ['生产时间']
    time = pd.to_datetime(time.iloc[:,0])

    df.index = time

    df = df.resample(rule).first()
    return df

In [ ]:
%%time

filelist = [r'D:\datasets\宁波20181213-20190110\2018-12-13_22-45_89_liqun89.dat',
            r'D:\datasets\宁波20181213-20190110\2018-12-14_7-30_90_liqun89.dat',
            r'D:\datasets\宁波20181213-20190110\2018-12-14_22-45_91_liqun89.dat',
            r'D:\datasets\宁波20181213-20190110\2018-12-15_7-30_92_liqun89.dat']
dflist = []

for f in filelist:
    df = gendf(f)
    dflist.append(df)

In [ ]:
dflist[0].head()

In [ ]:
dflist[1].head()

In [ ]:
dflist[2].head()

In [ ]:
dflist[3].head()

In [ ]:
# dflist[0].to_csv('dflist[0].csv', encoding='gbk')

In [ ]:
# dflist[1].to_csv('dflist[1].csv', encoding='gbk')

In [ ]:
# dflist[2].to_csv('dflist[2].csv', encoding='gbk')

In [ ]:
# dflist[3].to_csv('dflist[3].csv', encoding='gbk')

In [ ]:
dflist[0].吸丝带位置.plot(figsize=(12,6))
plt.ylabel('吸丝带位置')
plt.show()

In [ ]:
dflist[1].吸丝带位置.plot(figsize=(12,6))
plt.ylabel('吸丝带位置')
plt.show()

In [ ]:
dflist[2].吸丝带位置.plot(figsize=(12,6))
plt.ylabel('吸丝带位置')
plt.show()

In [ ]:
dflist[3].吸丝带位置.plot(figsize=(12,6))
plt.ylabel('吸丝带位置')
plt.show()

In [ ]:
dflist[0].describe()

In [ ]:
dflist[1].describe()

In [ ]:
dflist[2].describe()

In [ ]:
dflist[3].describe()

In [ ]:
plt.boxplot(dflist[3].吸丝带位置.dropna())

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

X = scaler.fit_transform(dflist[3].iloc[:,3:6].dropna())

res = pd.DataFrame(X).diff().dropna().apply(lambda x: sum(x**2), axis=1)
res[:200].plot()